# Kirsh (2009), Fig. 3
In this example we reproduce the results of Kirsh et al. (2009) Fig. 3, using a 2.3$M_{\oplus}$ planet embedded in a planetesimal disk undergoing migration.

In [1]:
import rebound
import numpy as np
import time as timing
import os

## Setup
First we need to setup the problem with all the various functions we'll need.

In [2]:
#various powerlaws we'll need
def rand_powerlaw(slope, min_v, max_v):
    y = np.random.uniform()
    pow_max = pow(max_v, slope+1.)
    pow_min = pow(min_v, slope+1.)
    return pow((pow_max-pow_min)*y + pow_min, 1./(slope+1.))

def rand_uniform(minimum, maximum):
    return np.random.uniform()*(maximum-minimum)+minimum

def rand_rayleigh(sigma):
    return sigma*np.sqrt(-2*np.log(np.random.uniform()))

In [3]:
#calculating the semi-major axis
def get_a(sim,planet_index):
    particles = sim.particles
    com = sim.calculate_com()
    p = particles[planet_index]
    mu = sim.G*(com.m+p.m)
    dvx = p.vx - com.vx
    dvy = p.vy - com.vy
    dvz = p.vz - com.vz
    v2 = dvx*dvx + dvy*dvy + dvz*dvz
    dx = p.x - com.x
    dy = p.y - com.y
    dz = p.z - com.z
    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return -mu/(v2 - 2.*mu/r)

In [4]:
#The outputs for the integrator
def get_outputs(sim,E0,start_t):
    dE = abs((sim.calculate_energy() - E0)/E0)
    current_t = timing.time()-start_t
    a = get_a(sim,sim.N_active-1)
    N_mini = 0
    #if(sim.integrator == "hybarid"):
    #    N_mini = sim.ri_hybarid.mini.N   #no attribute .mini.N?
    return sim.t, dE, a, sim.N, N_mini, current_t, sim.ri_hybarid.timestep_too_large_warning

In [23]:
#The Kirsh problem
def problem(HSR, dt, seed, run):
    
    #output initialization stuff
    filename = "output/Kirsh_HSR"+str(HSR)+"_dt"+str(dt)+"_"+str(run)+".txt"
    try:
        os.remove(filename)
    except OSError:
        pass
    np.random.seed(seed=seed)
    write_t = dt      #writing to file: ini
    write_log = 1.01  #writing to file: log output
    write_lin = 100   #writing to file: linear output every write_lin years

    #initialize the simulation
    sim = rebound.Simulation()
    
    #integrator options
    sim.integrator = "hybarid"
    sim.ri_hybarid.switch_radius = HSR
    sim.ri_hybarid.CE_radius = 15.
    sim.dt = dt
    sim.testparticle_type = 1
    sim.usleep = -1
    tmax = 1e5*6.283
    #tmax = 1000*6.283
    
    #collision and boundary options
    sim.collision = "direct"
    sim.collision_resolve = "merge"
    sim.boundary = "open"
    boxsize = 80.
    sim.configure_box(boxsize)
    sim.collisions_track_dE = 1
    
    #add sun
    sim.add(m=1.)
    
    #Super Earth
    m_earth = 0.000003003
    m_planet = m_earth*2.3
    a_planet = 25
    #r_planet = 0.0000788215 #radius of particle using 2g/cm^3 (AU)
    r_planet = 4.68e-4 #Jupiter inflated mass
    sim.add(m=m_planet,r=r_planet,a=a_planet,e=0,inc=0.0001)
    
    sim.N_active = sim.N
    
    #planetesimal disk
    print "Populating planetesimal disk for run %d"%run
    m_pl = m_planet/600.
    N_pl = 230*m_earth/m_pl
    r_pl = 0.00000934532
    disk_min_a = a_planet - 10.5
    disk_max_a = a_planet + 10.5
    while sim.N < (N_pl + sim.N_active):
        a = rand_powerlaw(1, disk_min_a, disk_max_a)
        e = rand_rayleigh(0.01)
        inc = rand_rayleigh(0.05)
        Omega = rand_uniform(0,2*np.pi)
        omega = rand_uniform(0,2*np.pi)
        f = rand_uniform(0,2*np.pi)
        sim.add(m=m_pl, r=r_pl, a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f) #This is very(!) slow
    
    #final physics initialization stuff
    sim.move_to_com()
    E0 = sim.calculate_energy()
    
    print "Begin simulation for run %d"%run
    start_t = timing.time()
    while sim.t < tmax:
        sim.integrate(write_t)
        with open(filename, "a") as writefile:
            writefile.write(','.join(map(str, get_outputs(sim,E0,start_t))) +"\n")
        write_t = min(write_t*write_log,write_t*write_lin)
        if write_t < sim.t + sim.dt:
            write_t = sim.t + sim.dt
        
    print("Run %d complete in %.2f s with dE=%e."% (run,timing.time()-start_t,abs((sim.calculate_energy() - E0)/E0)))

## Run the problem
Now let's run the problem, making use of the multiprocessing module.

In [ ]:
#Setup parameters
dt = 12.76
HSR = 3
n_runs = 6
params = []
for i in range(n_runs):
    params.append((HSR,dt,int(np.random.random()*1000),i))

**Warning!** executing the code below will take a long time!

In [ ]:
#Run the problem
import multiprocessing as mp
def execute(pars):
    problem(pars[0],pars[1],pars[2],pars[3])

if __name__== '__main__':
    pool = mp.Pool(processes=len(params))
    pool.map(execute, params)
    pool.close()
    pool.join()

## Plot the results
It's time to plot our results, which were saved in .txt files. 

In [ ]:
def get_file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [ ]:
import matplotlib.pyplot as plt
import glob
%matplotlib inline
filenames = "output/Kirsh_HSR"+str(HSR)+"_dt"+str(dt)
files = glob.glob(filenames+"_*.txt")
fn = len(files)

#find array with shortest length
flen = 1e10
for f in files:
    flen = min(flen,get_file_len(f))

#read in data 
data = []
for f in files:
    try:
        ff = open(f, 'r')
        lines = ff.readlines()
        data.append(lines)
    except:
        print "couldnt read in data file "+f

#make arrays we'll need
E = np.zeros(shape=(fn,flen))
Eavg = np.zeros(flen)
a = np.zeros(shape=(fn,flen))
aavg = np.zeros(flen)
Navg = np.zeros(flen)
time = np.zeros(flen)

#extract data
for i in xrange(0,flen):
    E_avg = np.zeros(fn)
    a_avg = np.zeros(fn)
    N_avg = np.zeros(fn)
    for j in xrange(0,fn):
        split = data[j][i].split(",")
        E_avg[j] = float(split[1])
        a_avg[j] = float(split[2])
        N_avg[j] = float(split[3])
        E[j][i] = E_avg[j]
        a[j][i] = a_avg[j]
    Eavg[i] = np.mean(E_avg)
    aavg[i] = np.mean(a_avg)
    Navg[i] = np.mean(N_avg)
    time[i] = float(split[0])/6.283185

#setup plot
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10,10))
alpha = 0.5
mint = min(time)
maxt = max(time)
axes[0].set_yscale('log')
axes[0].set_xscale('log')
axes[0].set_ylabel('dE')
axes[0].set_xlim([mint,maxt])
axes[1].set_xscale('log')
axes[1].set_ylabel('Number of bodies')
axes[1].set_xlim([mint,maxt])
axes[2].set_ylabel('Semi-major axis')
axes[2].set_xlabel('time (years)')

#plot
for i in xrange(0,fn):
    axes[0].plot(time,E[i], '.', alpha=alpha, color='lightgreen')
    axes[2].plot(time,a[i], '.', alpha=alpha, color='lightblue')
axes[0].plot(time, Eavg, '.', markeredgecolor='none', color='green', label='avg.')
axes[1].plot(time, Navg, '.', markeredgecolor='none', color='black', label='avg.')
axes[2].plot(time, aavg, '.', markeredgecolor='none', color='blue', label='avg.')
plt.savefig(filenames+"_avg.pdf") #save plot